In [1]:
import os
import pandas as pd

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
absolute_path = '/Users/aygalic/OneDrive/polimi/Thesis/data/quant/'

# Exploring the Dataset

In [3]:
# getting entries ready
# each couple of entries correspond to one patient

entries = os.listdir(absolute_path)


In [4]:
entries_transcripts = [e for e in entries if "transcripts" in e ]

# Building a TensorFlow input pipeline

## We want to build a tf.Dataset from this

In [5]:
# from filename to tensor

def load_patient_data(filename):
  #specify read types for our data
  read_types = [ float()]
  # get a first sample to base everything of
  text = pathlib.Path(absolute_path + filename).read_text()
  lines = text.split('\n')[1:-1]
  features = tf.io.decode_csv(lines, record_defaults=read_types, field_delim = "\t", select_cols=[3]) 
  data = tf.convert_to_tensor(features)

  return data

# Feed it into a net ?

In [6]:
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

# now we build a tf.dataset with all patients inside


In [7]:
# load the dataset into a list using the first pipeline

train_ds = [load_patient_data(e) for e in entries_transcripts]
train_ds = [e for e in train_ds if e.shape == (1, 95309)]


In [8]:
train_ds[0]


<tf.Tensor: shape=(1, 95309), dtype=float32, numpy=
array([[ 0.     ,  6.55631,  0.     , ..., 21.601  , 14.5302 , 43.2891 ]],
      dtype=float32)>

In [9]:
# turn it into a tf.data.Dataset object
x_train = tf.data.Dataset.from_tensor_slices(train_ds)



In [10]:
# input are the same as the target
zipped_boi = tf.data.Dataset.zip((x_train, x_train))


In [11]:
#(iter(zipped_boi))

# Load the model

In [13]:
model = keras.models.load_model(
    "../workfiles/placeholder_model/"
)


# compute the output

In [14]:
# Try on a single file and check the result.
compressed_0 = model.encoder.predict(train_ds[0])
print(compressed_0[0])

reconstruct_0 = model.decoder.predict(compressed_0)
print(reconstruct_0[0][0])


1/1 [==============================] - 0s 181ms/step


2023-07-05 18:44:46.647661: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[    0.     13090.774      0.         0.         0.      7381.1904
     0.      2009.5237     0.         0.     35305.48       0.
     0.         0.         0.         0.         0.     10935.378
     0.      8811.2     4713.6577     0.         0.         0.
 18364.182   2599.1255  6657.475      0.         0.     20994.81
  5522.911      0.         0.         0.         0.         0.
  5593.051      0.         0.     12985.868  29643.748      0.
     0.      8116.364      0.      3961.9128  7013.927      0.
     0.      5683.1914 11199.816      0.     11859.392  16064.646
     0.      7418.711  10018.981      0.         0.      4571.481
 10071.145   9772.012      0.         0.    ]
1/1 [==============================] - 0s 49ms/step
[0. 0. 0. ... 0. 0. 0.]


In [15]:
# now let's do all of it with list comprehension
compressed_dataframe = [model.encoder.predict(sample)[0] for sample in train_ds]


1/1 [==============================] - 0s 27ms/step


# now we have to match the corresponding file names with the compressed output.


In [16]:
train_ds_ = [load_patient_data(e) for e in entries_transcripts]



In [17]:
samples_to_keep = [1 if s.shape ==(1, 95309) else 0 for s in train_ds_]
filenames = [f for (f, test) in  zip(entries_transcripts, samples_to_keep) if test]

print(len(filenames))
print(len(compressed_dataframe))

4755
4755


In [18]:
df = pd.DataFrame(compressed_dataframe)
df["name"] = filenames


In [19]:
df.to_csv("../workfiles/compressed_data.csv")